# Inventory Management Environments

> To be written.

In [ ]:
#| default_exp dataloaders.tabular

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import numpy as np
from abc import ABC, abstractmethod
from typing import Union

from ddopnew.dataloaders.base import BaseDataLoader

In [ ]:
#| export
class XYDataLoader(BaseDataLoader):
    
    def __init__(self,
        X: np.ndarray,
        Y: np.ndarray,
        val_index_start: Union[int, None] = None, # give list
        test_index_start: Union[int, None] = None, # give list
    ):
        self.X = X
        self.Y = Y

        self.val_index_start = val_index_start
        self.test_index_start = test_index_start

        if self.val_index_start is not None:
            self.train_index_end = self.val_index_start-1
        elif self.test_index_start is not None:
            self.train_index_end = self.test_index_start-1
        else:
            self.train_index_end = len(Y)-1

        self.dataset_type = "train"

        if len(X.shape) == 1:
            self.X = X.reshape(-1, 1)
        
        if len(Y.shape) == 1:
            self.Y = Y.reshape(-1, 1)

        assert len(X) == len(Y), 'X and Y must have the same length'

        self.num_units = Y.shape[1] # shape 0 is alsways time, shape 1 is the number of units (e.g., SKUs)

        super().__init__()
    
    def __getitem__(self, idx): 

        if self.dataset_type == "train":

            if idx > self.train_index_end:
                raise IndexError(f'index{idx} out of range{self.train_index_end}')
            idx = idx
        elif self.dataset_type == "val":
            idx = idx + self.val_index_start
            
            if idx >= self.test_index_start:
                raise IndexError(f'index{idx} out of range{self.test_index_start}')
            
        elif self.dataset_type == "test":
            idx = idx + self.test_index_start
            
            if idx >= len(self.X):
                raise IndexError(f'index{idx} out of range{len(self.X)}')
        
        else:
            raise ValueError('dataset_type not set')

        return self.X[idx], self.Y[idx]

    def __len__(self):
        return len(self.X)
    
    @property
    def X_shape(self):
        return self.X.shape
    
    @property
    def Y_shape(self):
        return self.Y.shape

    @property
    def len_train(self):
        return self.train_index_end+1

    @property
    def len_val(self):
        if self.val_index_start is None:
            raise ValueError('no validation set defined')
        return self.test_index_start-self.val_index_start

    @property
    def len_test(self):
        if self.test_index_start is None:
            raise ValueError('no test set defined')
        return len(self.X)-self.test_index_start

    def get_all_X(self):

        """
        Returns the entire features dataset. If no X data is available, return None.
        """
        return self.X.copy() if self.X is not None else None

    def get_all_Y(self):

        """
        Returns the entire target dataset. If no Y data is available, return None.
        """
        return self.Y.copy()if self.Y is not None else None

In [ ]:
X = np.random.standard_normal((100, 2))
Y = np.random.standard_normal((100, 1))
Y += 2*X[:,0].reshape(-1, 1) + 3*X[:,1].reshape(-1, 1)

dataloader = XYDataLoader(X = X, Y = Y)

sample_X, sample_Y = dataloader[0]
print("sample:", sample_X, sample_Y)
print("sample shape Y:", sample_Y.shape)

print("length:", len(dataloader))

sample: [1.20283467 0.41742205] [4.31383166]
sample shape Y: (1,)
length: 100


In [ ]:
X = np.random.standard_normal((10, 2))
Y = np.random.standard_normal((10, 1))
Y += 2*X[:,0].reshape(-1, 1) + 3*X[:,1].reshape(-1, 1)

dataloader = XYDataLoader(X = X, Y = Y, val_index_start=6, test_index_start=8)

sample_X, sample_Y = dataloader[0]

print("length train:", dataloader.len_train, "length val:", dataloader.len_val, "length test:", dataloader.len_test)

print("")
print("### Data from train set ###")
for i in range(dataloader.len_train):
    sample_X, sample_Y = dataloader[i]
    print("idx:", i, "data:", sample_X, sample_Y)

dataloader.val()

print("")
print("### Data from val set ###")
for i in range(dataloader.len_val):
    sample_X, sample_Y = dataloader[i]
    print("idx:", i, "data:", sample_X, sample_Y)

dataloader.test()

print("")
print("### Data from test set ###")
for i in range(dataloader.len_test):
    sample_X, sample_Y = dataloader[i]
    print("idx:", i, "data:", sample_X, sample_Y)

length train: 6 length val: 2 length test: 2

### Data from train set ###
idx: 0 data: [ 0.55353506 -0.65329557] [-1.5408286]
idx: 1 data: [-1.19035247 -0.25282667] [-1.62284877]
idx: 2 data: [-0.8080214 -0.0018918] [-0.47407762]
idx: 3 data: [-0.91445465  0.07642132] [-0.8490106]
idx: 4 data: [1.49728699 0.16848675] [2.44198599]
idx: 5 data: [-1.95742975 -1.72847056] [-10.3822297]

### Data from val set ###
idx: 0 data: [ 0.55353506 -0.65329557] [-1.5408286]
idx: 1 data: [-1.19035247 -0.25282667] [-1.62284877]

### Data from test set ###
idx: 0 data: [ 0.55353506 -0.65329557] [-1.5408286]
idx: 1 data: [-1.19035247 -0.25282667] [-1.62284877]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()